### 01. `Autogluon` 라이브러리 설치

In [ ]:
!pip install autogluon

### 2. 필요한 라이브러리 추가

In [ ]:
# built-in
import os
import random

# basic external
import numpy as np
import pandas as pd

# AutoML
from autogluon.tabular import TabularDataset, TabularPredictor

### 3. 필요한 파라미터 정의

In [ ]:
exp_hparams = {
    'seed': 42,
    'tr_data_path': '/kaggle/input/aimers06/data/train.csv',
    'tt_data_path': '/kaggle/input/aimers06/data/test.csv',
    'sub_data_path': '/kaggle/input/aimers06/data/sample_submission.csv',
}

In [ ]:
autogluon_hparams = {
    'label': "임신 성공 여부", # 정답 컬럼명
    'problem_type': 'binary', # 풀고자 하는 문제의 유형
    'eval_metric': "roc_auc", # 평가 지표
    'verbosity': 3, # 출력 레벨
}

### 4. 필요 함수 정의

In [ ]:
def set_seed(seed: int = 2025) -> None:
    """실험 재현을 위해 seed를 설정하는 함수입니다.

    Args:
        seed (int, optional): seed로 사용할 정수형 값. Defaults to 2025.
    """
    random.seed(seed)
    os.environ['PYTHONASHSEED'] = str(seed)
    np.random.seed(seed)

### 5. 사전 작업

In [ ]:
# 시드 고정
set_seed(exp_hparams['seed'])

In [ ]:
# 데이터 로드(ID 컬럼 제외)
tr_df = pd.read_csv(exp_hparams['tr_data_path']).drop(columns=['ID'])
tt_df = pd.read_csv(exp_hparams['tt_data_path']).drop(columns=['ID'])
sub_df = pd.read_csv(exp_hparams['sub_data_path'])

### 6. Autogluon 실행

In [ ]:
# predictor 정의
predictor = TabularPredictor(
    **autogluon_hparams
)

In [ ]:
# AutoML 실행
predictor.fit(
    train_data=TabularDataset(tr_df),
    presets='best_quality', # default
)

### 7. AutoML 결과 확인

In [ ]:
predictor.leaderboard()

### 8. submission 파일 생성

In [ ]:
sub_df['probability'] = predictor.predict_proba(
    data=tt_df,
)[:][1]
sub_df.to_csv("presets_best.csv", index=False)